In [1]:
from sentinelhub import (
    SHConfig,
    BBox,
    bbox_to_dimensions,
    DataCollection,
    MimeType,
    MosaickingOrder,
    SentinelHubRequest,
    BBoxSplitter
)

In [2]:
def get_images(coordinates):
    
    # put here your sentinel hub credential (these expires on 30 June 2024)
    config = SHConfig()
    config.sh_client_id = "deb35d4d-5181-47d5-8751-ed5b4e478313"
    config.sh_client_secret = "gOVrFxE3li6RNS0AG5NCbXpcYCp5TeuY"

    # customize this functions to specify the band you want to download
    evalscript = """
        // VERSION=3
        
        function setup() {
            return {
                input: [{
                    bands: [
                        "B02",
                        "B03",
                        "B04",
                        "B08"
                    ],
                    units: "DN"  // digital number
                }],
                output: {
                    bands: 4,
                    sampleType: "UINT8"  // 0 - 255
                }
            };
        }

        function evaluatePixel(sample) {
            return [
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B08
            ];
        }
    """

    # put here the coordinate of the image you want to download
    # coordinates = (-96.455, 41.146, -96.450, 41.153)

    # set the coordinate reference system
    crs = 4326

    # set the desired resolution
    resolution = 10

    # set the time interval
    time_interval = ("2022-08-01", "2022-08-31")

    # set the source of the image
    data_collection = DataCollection.SENTINEL2_L2A

    # set the constant class for type of mosaicking order
    mosaicking_order = MosaickingOrder.LEAST_CC

    # update this value if you want to save the image
    save_data = True

    # set the folder where you want to dowload the image
    data_folder = './immagini'

    # set the format of the image
    response_format = MimeType.TIFF

    bbox = BBox(bbox = coordinates, crs = crs)
    size = bbox_to_dimensions(bbox, resolution = resolution)
    request = SentinelHubRequest(
        config = config,
        bbox = bbox,
        size = size,
        evalscript = evalscript,
        data_folder = data_folder,
        input_data = [
            SentinelHubRequest.input_data(
                time_interval = time_interval,
                data_collection = data_collection,
                mosaicking_order = mosaicking_order
            )
        ],
        responses = [
            SentinelHubRequest.output_response(
                identifier = "default",
                response_format = response_format
            )
        ]
    )

    return request.get_data(save_data = save_data)

In [ ]:
import math

def divide_bbox(bbox:BBox, resolution:float, max_size=2400):
    """
        This function allows to divide a bounding box into smaller pieces.
    """
    dimensions = bbox_to_dimensions(bbox, resolution)
    if dimensions[0] > max_size or dimensions[1] > max_size:
        x_divisions = max(1, math.ceil(dimensions[0] / max_size))
        y_divisions = max(1, math.ceil(dimensions[1] / max_size))
        print(y_divisions, x_divisions)
        bbox_splitter = BBoxSplitter([bbox], 4326, (x_divisions, y_divisions))
        return bbox_splitter.get_bbox_list()
    else:
        return [bbox]


x1 = -116
x2 = -114.5
y1 = 40
y2 = 41

cont = 0
for bbox in divide_bbox(BBox((x1, y1, x2, y2), 4326), 10):
    get_images(bbox)
    cont += 1
    print(cont)

In [4]:
import folium
import os
import rasterio

map = folium.Map()
for path in os.listdir('immagini'):
    with rasterio.open(f'immagini/{path}/response.tiff') as image:
        left, bottom, right, top = image.bounds
        folium.Rectangle([(top, left), (bottom, right)], color='black', weight=1).add_to(map)
map